In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [31]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # in the conv2D function we have to specify the channels
        # Since the image here has 3 channels RGB we hare setting it to 3 here
        self.conv1 = nn.Conv2d(3, 6, 5)
        # polling window will be of the size 2*2
        # and After ech operation the pooling window will shift by 2 pixels
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # To flatten the image since there are 16 channels that means 16 different 
        # filtered images each with a dimension of 5 * 5 so flattened input will have 16*5*5
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        y = self.fc3(x)
        
        return y

In [32]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
num_epochs = 5
batch_size = 4
learning_rate = 0.001

# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [33]:
# demonstration of how convalutions are applied and how the images is shrunk
conv1 = nn.Conv2d(3, 6, 5)
pool = nn.MaxPool2d(2,2)
conv2 = nn.Conv2d(6, 16, 5)

examples = iter(train_loader)
images, lables = next(examples)
print(images.shape)

# convalutions applies filter to the image this causes the image to shrink
x = conv1(images)
print(x.shape)

# pooling will reduce the image size by a factor of 2
x = pool(x)
print(x.shape)
x = conv2(x)
print(x.shape)
x = pool(x)
print(x.shape)

torch.Size([4, 3, 32, 32])
torch.Size([4, 6, 28, 28])
torch.Size([4, 6, 14, 14])
torch.Size([4, 16, 10, 10])
torch.Size([4, 16, 5, 5])


In [34]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1/5], Step [2000/12500], Loss: 2.3192
Epoch [1/5], Step [4000/12500], Loss: 2.2975
Epoch [1/5], Step [6000/12500], Loss: 2.3183
Epoch [1/5], Step [8000/12500], Loss: 2.0790
Epoch [1/5], Step [10000/12500], Loss: 1.6282
Epoch [1/5], Step [12000/12500], Loss: 1.8311
Epoch [2/5], Step [2000/12500], Loss: 2.2330
Epoch [2/5], Step [4000/12500], Loss: 1.7487
Epoch [2/5], Step [6000/12500], Loss: 2.5471
Epoch [2/5], Step [8000/12500], Loss: 1.9375
Epoch [2/5], Step [10000/12500], Loss: 1.7506
Epoch [2/5], Step [12000/12500], Loss: 1.9187
Epoch [3/5], Step [2000/12500], Loss: 1.2373
Epoch [3/5], Step [4000/12500], Loss: 2.0763
Epoch [3/5], Step [6000/12500], Loss: 2.5612
Epoch [3/5], Step [8000/12500], Loss: 2.2818
Epoch [3/5], Step [10000/12500], Loss: 3.0536
Epoch [3/5], Step [12000/12500], Loss: 1.3392
Epoch [4/5], Step [2000/12500], Loss: 0.6526
Epoch [4/5], Step [4000/12500], Loss: 1.7616
Epoch [4/5], Step [6000/12500], Loss: 1.4755
Epoch [4/5], Step [8000/12500], Loss: 1.4994
Epoc